In [1]:
import sys
from pathlib import Path

# Find project root and add 'src' folder to sys.path
project_root = Path.cwd().parents[0]  # goes from /notebooks/ to project root
src_path = project_root / 'src'
sys.path.append(str(src_path))

In [7]:
from data_fetching.fetch_yfinance import get_price_data

tickers = ['AAPL', 'TSLA', 'NVDA']
start_date = '2024-01-01'
end_date = '2025-01-01'

prices = get_price_data(tickers, start_date, end_date)
print(prices.info)

[*********************100%***********************]  3 of 3 completed

<bound method DataFrame.info of Ticker            AAPL        NVDA        TSLA
Date                                          
2024-01-02  184.290421   48.149918  248.419998
2024-01-03  182.910507   47.551144  238.449997
2024-01-04  180.587524   47.979984  237.929993
2024-01-05  179.862854   49.078568  237.490005
2024-01-08  184.210999   52.233379  240.449997
...                ...         ...         ...
2024-12-24  257.578674  140.207108  462.279999
2024-12-26  258.396667  139.917130  454.130005
2024-12-27  254.974930  136.997391  431.660004
2024-12-30  251.593094  137.477356  417.410004
2024-12-31  249.817383  134.277649  403.839996

[252 rows x 3 columns]>
